In [5]:
import os

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow import keras
from random import randint

# https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch
# https://www.tensorflow.org/tutorials/generative/autoencoder

# disable gpu (only enable cpu)
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

class NumberVectorGenerator:

    @staticmethod
    def generate( shape : tuple ) -> np.ndarray:
        return np.random.randint(low=0, high=255, size=shape)

class SimpleAutoEncoder(keras.Model):

    def __init__(self, shape):
        super(SimpleAutoEncoder, self).__init__()

        self.shape = shape
        print(shape)
        self.encoder = keras.Sequential([
            keras.layers.Flatten(name='flatten'),
            keras.layers.Dense(shape[0]*shape[1], activation='relu', name='dense_1'),
            keras.layers.Dense(25*25*3, activation='relu', name='dense_2'),
            keras.layers.Dense(10*10*3, activation='relu', name='dense_3'),
            keras.layers.Dense(3*3*3, activation='relu', name='dense_4'),
        ], name="encoder")
        self.encoder.build(input_shape=shape)
    
        self.decoder = keras.Sequential([
            keras.layers.Dense(3*3*3, activation='relu', name='dense_1'),
            keras.layers.Dense(10*10*3, activation='relu', name='dense_2'),
            keras.layers.Dense(25*25*3, activation='relu', name='dense_3'),
            keras.layers.Dense(shape[0]*shape[1], activation='sigmoid', name='dense_4'),
            keras.layers.Reshape(target_shape=shape, name='reshape'),
        ], name="decoder")
        self.decoder.build(input_shape=(1,3*3*3))

    def call( self, x ) -> tuple:
        # print( np.shape(x) )
        encoded = self.encoder( x )
        # print( np.shape(encoded) )
        decoded = self.decoder(encoded)
        # print( np.shape(decoded) )
        return decoded

_shape = (50*50, 3)

autoencoder = SimpleAutoEncoder(shape=_shape)
autoencoder.compile(optimizer='adam', loss=keras.losses.MeanSquaredError())

print(autoencoder.encoder.summary(), autoencoder.decoder.summary())

data = NumberVectorGenerator.generate( (1000, _shape[0], _shape[1]) )
print( np.shape(data), data[randint(1, len(data)-1 )] )

# split training set
x_train, x_test = train_test_split( data, test_size=0.3, shuffle=True )
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# fit
history = autoencoder.fit(
    x_train, x_train,
    epochs=12,
    shuffle=True,
    batch_size=32,
    validation_data=(x_test, x_test),
    use_multiprocessing=True
)

encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

# n = 10
# plt.figure(figsize=(20, 4))
# for i in range(n):
#     # display original
#     ax = plt.subplot(2, n, i + 1)
#     plt.imshow(x_test[i])
#     plt.title("original")
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)

#     # display reconstruction
#     ax = plt.subplot(2, n, i + 1 + n)
#     plt.imshow(decoded_imgs[i])
#     plt.title("reconstructed")
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)
# plt.show()

plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()

(2500, 3)
Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (2500, 3)                 0         
                                                                 
 dense_1 (Dense)             (2500, 7500)              30000     
                                                                 
 dense_2 (Dense)             (2500, 1875)              14064375  
                                                                 
 dense_3 (Dense)             (2500, 300)               562800    
                                                                 
 dense_4 (Dense)             (2500, 27)                8127      
                                                                 
Total params: 14665302 (55.94 MB)
Trainable params: 14665302 (55.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: 

ValueError: in user code:

    File "/home/user/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/home/user/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/user/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/home/user/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/src/engine/training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "/home/user/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_f

In [ ]:
import os
import numpy as np
import json

from PIL import Image
from typing import Any

def slice_to_chunks( array : list, chunk_size : int ) -> list:
    for i in range(0, len(array), chunk_size):  
        yield array[i:i + chunk_size]

def split_into_chunks( pixel_array : list, chunk_size : int, blank_values : Any ) -> list:
    chunks = list( slice_to_chunks( pixel_array, chunk_size ) )
    chunks[-1].extend([ blank_values ] * (chunk_size - len( chunks[-1] )))
    return chunks

def encode_image( img : Image.Image ) -> np.ndarray:
    # image dimensions
    print( "Input Image Shape;", img.size[::-1] )
    
    # removes alpha by using only RGB
    img = img.convert('RGB')
    
    # read the pixels from the image
    pixels_d2 = np.array(img).tolist()
    
    # get the complete flattened pixel array
    pixel_array = []
    for col in pixels_d2:
        pixel_array.extend(col)
    print( "Total Pixels; ", len(pixel_array))

    #chunk the pixels
    print("Chunking Arrays")
    chunked = np.array( split_into_chunks( pixel_array, _shape[0], [0,0,0] ) )
    print("Chunked Arrays: ", len(chunked))
    
    # encode the values
    print("Encoding")
    encoded_values = autoencoder.encoder([chunked]).numpy().tolist()
    print("Encoded")

    # return the encoded values list
    return encoded_values

def decode_image( values : np.ndarray ) -> Image.Image:
    pass # decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

img = Image.open("/mnt/c/Users/Declan/Downloads/31tkCSZqN3L_2.png")
print(img.size)

encoded = encode_image( img )
print(np.shape(encoded))

with open("/mnt/c/Users/Declan/Downloads/out.json", "w") as file:
    file.write( json.dumps(encoded) )

# decoded = decode_image( encoded )
# print(decoded.size)
